# Einführung in das Control-Modul von pandapower

Das Control-Modul in pandapower ermöglicht es Ihnen, einzelne Komponenten innerhalb eines Stromnetzes anzupassen, zu regeln und zu manipulieren. Die Möglichkeiten hierbei sind vielfältig und unerliegen rein Ihrer Vorstellungskraft. Sie reichen von einfachen Reglern, die es bspw. ermöglichen im Zuge einer Zeitreihensimulation die Einträge von Lasten, Einspeiser oder Speichern anzupassen und zu manipulieren, über betriebsmittelspezifische Regelungen wie Stufensteller bis hin zu komplexen Regler, die die verschiedenen Flexibilitäten innerhalb eines Stromnetzes ansteuern. 

Dieses Tutorial soll Ihnen dabei helfen, ein grundlegendes Verständnis des Control-Moduls zu erhalten und Sie damit in die Lage versetzen, wie Sie das Control-Modul für Ihre Zwecke verwenden können.

## Struktur des Control-Moduls

Das Controll-Modul ist im Vergleich zu den anderen Komponenten, aus denen sich ein Stromnetz aufbaut, eher ein Exokt. Denn anderes als bei allen anderen Komponenten mussten wir bei den Reglern einsehen, dass es hier weniger sinnvoll ist, von der klassischen Tabellen-Struktur auszugehen und zu versuchen, die Regler in eine fixe Struktur einzubetten. Anders gesagt: Während Busse und Leitungen immer von fixen Größen abhängen, so gilt das im Falle von Reglern nicht: Regler sind ungemein flexibel. Was zum einen deren Stärke ist (die Möglichkeiten sind hier grenzenlos), sorgt zum anderen für den Verlust der übersichtlichen Tabellenstruktur, die Sie von den anderen bereits kennengelernten Komponenten kennen.

Das soll allerdings nicht davon abhalten, unser Control-Modul nach Ihrem Belieben einzusetzen. Hierzu haben wir versucht, es so strukturiert wie möglich zu gestalten, damit Sie sich schnell zurecht finden und ohne Probleme Ihre eigenen Regler basteln können.

### Netz-Einbettung der Regler

Jeder Regler, der in ein pandapower-Netz integriert wird, ist ein Objekt. Jedes Regler-Objekt basiert auf der abstrakten Klasse Controller in der basic_controller.py. Für einen Regler wird immer ein Netz benötigt, dem der Regler zugeordnet ist. 

In [ ]:
import pandapower as pp

net = pp.create_empty_network()

Der Basiscontroller braucht als externe Variable lediglich das Netz:

In [ ]:
from pandapower.control.basic_controller import Controller

In [ ]:
basic_control = Controller(net)

Ein Blick auf die einzelnen Attribute verrät, was elementare Teile eines jeden Reglers sind:

In [ ]:
basic_control.__dict__


Das Attribut 'recycle' sollten Sie nur ändern, wenn Sie wirklich wissen, was Sie tun. Hierbei geht es um die Beschleunigung bei aufeinderfolgenden Simulationen wie bspw. Zeitreihensimulationen. 

Die Flag 'recycle' zeigt an, ob bei jeder einzelnen Simulation die Jacobimatrix neu aufgebaut werden muss (recycle = False) oder nicht. Sind Sie sich also ABSOLUT sicher, dass sich die Struktur Ihres Netzes nicht verändert, sondern letztlich nur die Werte innerhalb der Jacobimatrix angepasst werden müssen, dann können Sie, um Ihre Berechnungen zu boosten, 'recycle' auf True setzen. Doch Obacht, schnell kann hier etwas schief gehen und folgich Ihre Ergebnisse überhaupt nicht das zurückspielen, was Sie eigentlich beabsichtigt hatten.

Das Attribut 'initial_run' gibt an, ob vor der eigentlichen Regelung ein Lastfluss durchgeführt werden muss (initial_run = True) oder nicht. Das ist bspw. relevant, wenn der Regler Werte aus den Lastflussergebnissen als Input benötigt.

Der 'index' ist die Indizierung innerhalb des Netzes, sozusagen, der Ort, wo Sie unter net.controller Ihren Regler wiederfinden. Bei dem Speicherort handelt es sich um ein pandas-DataFrame.

In [ ]:
net.controller

Den Index finden Sie wieder am Anfang einer jeden Zeile. Dieser wird automatisch zugewiesen, kann aber auch von Ihnen individuell festgelegt werden. Des Weiteren finden Sie hier weitere wichtige Informationen:

Die Information 'in_service' gibt Ihnen die Möglichkeit, einen Regler jeder Zeit zu ignorieren, ohne ihn löschen zu müssen. 

'order' und 'level' wiederum sind am besten grafisch erklärt. 
Angenommen Sie haben 5 Regler:

<img src="pics/controller.jpg" style="width: 1000px;"/>

Entsprechend ihres 'level' und ihrer 'order' werden diese nun in eine Reihenfolge gebracht. Wichtig hierbei, je höher 'level' und 'order' sind, desto später werden die Regler aktiv, d.h. Regler mit einer höheren 'level' und 'order' sind wichtiger, da sie auf die Regelung eines zuvor ausgeführten Reglers reagieren können. Am Beispiel von oben sähe das folgendermaßen aus:

<img src="pics/controller_structure.jpg" style="width: 600px;"/>


Regler innerhalb eines 'level' müssen stets alle konvergieren. Regler in unterschiedlichen 'level' sind unabhägnig voneinander. Ein Beispiel würde folgendermaßen aussehen. Angenommen es gäbe zwei Stufenregler, die auf die Spannung eines bestimmten Knotens regeln. Der eine Regler möchte, dass die Spannung an diesem Knoten stets unter 1 p.u. ist, während der andere genau das Gegenteil möchte. Auch wenn dieses Beispiel wohl kaum in der Realität zu finden ist, veranschaulicht es doch recht gut, den Unterschied zwischen 'level' und 'order'. Wären beide Regler im selben 'level', würde die Reglerschleife niemals konvergieren, da die Bedingungen beider Regler niemals erfüllt sein können. Wäre nun allerdings einer der Regler in einem höheren 'level', würde jede einzelne Reglerschleife wiederum problemlos eine Lösung finde, da diese beiden Regler unabhängig voneinander sind. Der eine Regler würde bspw. in seinem 'level' die Spannung unter 1 p.u. drücken, während der zweite das dann in seinem 'level' wieder rückgängig machen würde. 

### Grundsätzlicher Aufbau eines jeden Reglers

Die wichtigsten Funktionen eines jeden Reglers sind die folgenden:
- intialize_control
- control_step
- repair_control
- finalize_control

Jede dieser vier Funktionen sind letztlich für die Regelung relevant und beschreiben das Verhalten des Reglers im Netz. 'initialize_control' wird zu Beginn aufgerufen, bevor die Reglerschleifen durchlaufen werden. Bspw. könnten hier die intialen Werte von P und Q einer bestimmten Last/Einspeisung abgerufen werden. Der 'control_step' beschreibt die eigentliche Regelverhalten des Reglers. 'repair_control' ermöglicht es Ihnen, sollte aufgrund der Regelung das Netz nicht konvergieren, einmalig je Regelkreis eine Korrektur vorzunehmen. Bspw. könnte hier der Umgang mit auftretenden NaN-Werten beschrieben werden. 'finalize_control' wird zum Schluss durchgeführt. Ein typischen Beispiel ist hier, die converged-Flag wieder auf False zu setzen, damit im darauf anschließenden Regelkreis der Regler nicht aus Versehen ingoriert wird.

Weitere relevante Funktionen sind:
- is_converged
- set_recycle

'is_converged' beschreibt, unter welchen Bedingungen der Regler sich eingependelt hat, d.h. konvergiert ist. 'set_recycle' definiert, inwieweit die Jacobimatrix in aufeinanderfolgenden Reglerschleifen wiederverwendet werden darf.

Eine weitere wichtige Funktion ist time_step. Diese ist allerdings erst für die Zeitreihensimulation relevant und aus diesem Grund sei an dieser Stelle auf das Zeitreihensimulationstutorial verwiesen.

Dies sind die primär relevanten Funktionen eines jeden Reglers. Möchten Sie selbst einen Regler bauen, so müssen Sie stets diese Funktionen mitdenken.

## Benutzung des Control-Moduls

Dieses Beispiel, dass Ihnen das Control-Modul näher bringen soll, verwendet den Stufenregler (TapControl). Hierfür müssen Sie zunächst einmal das MV-Oberrehin-Netz, das zwei 110/220 kV Trafos enthält, laden:

In [ ]:
import pandapower as pp
from pandapower.networks import mv_oberrhein

net = mv_oberrhein()
net.trafo

Hier sehen Sie nun beide Trafos mit all ihren Eigenschaften. 

Als nächstes sollten Sie einmal einen Lastfluss rechnen, um zu sehen, welche Spannung ober- und unterseitig vom Trafo herrschen. Dies machen Sie über den Befehl pp.runpp(net). Lassen Sie sich direkt danach die Ergebnisse ausgeben: Einmal oberspannungsseitig (wenig spannend, da hier der Slack hängt) ...

In [ ]:
pp.runpp(net)
net.res_trafo.vm_hv_pu

... und einmal unterspannungsseitig:

In [ ]:
net.res_trafo.vm_lv_pu

Was Sie zudem an der net.trafo-Ausgabe sehen können: Beide Trafos enthalten einen Stufenstellen, die in der neutralen Position in der Stufe 0 sind und um 9 Stufen nach unten und nach oben verändert werden können.
In unserem Fall befinden sich beide Stufensteller in folgenden Positionen:


In [ ]:
net.trafo['tap_pos']

Die Stufensteller-Position verändert sich innerhalb eines Lastflusses nicht. Allerdings ist es möglich, mithilfe der Stufenregler das Ergebnis des Lastflusses in Abhängigkeit von der Knotenspannung unter- oder oberspannungseitig im Vorhinen zu beeinflussen.

### Diskrete Stufenregler


Dem diskrete Stufenregler (DiscreteTapControl) in pandapower wird zunächst die id des Trafos übergeben, der geregelt werden soll. Voreingestellt ist, dass die Spannung unterspannungsseitig überprüft wird. Des Weiteren wird ein Totband übergeben, in dem sich die Spannung an einem bestimmten Knoten bewegen darf. In unserem Fall definieren wir für den ersten Trafo im Oberrhein-Netz ein Totband von 0.99 und 1.01 p.u.

In [ ]:
import pandapower.control as control
trafo_controller = control.DiscreteTapControl(net=net, tid=114, vm_lower_pu=0.99, vm_upper_pu=1.01)

Wie oben bereits erwähnt, wird auch dieser Regler automatisch im Netz registriert:

In [ ]:
net.controller

Um die Regler zu aktivieren, müssen nun beim Aufruf von pp.runpp run_control = True gesetzt werden. Ein Blick auf die Erbenisse am Trafo unterspannungsseitig verrät, inwieweit die Regler aktiv geworden sind:

In [ ]:
pp.runpp(net, run_control=True)
net.res_trafo.vm_lv_pu

Wie man erkennen kann, hat sich die Spannung verringert und bewegt sich nun im vorgegeben Bereich. Überprüfen wir zudem die Position der Schalterstellung, so kann man feststellen, dass sich die Position von -2 auf -1 verändert hat

In [ ]:
net.trafo['tap_pos']

### Kontinuierlicher Stufenregler

Neben dem diskreten Trafo-Stufensteller gibt es auch die Möglichkeit, einen kontinuirlichen Stufenregler anzuwenden. Das Besondere hierbei ist, dass man keinen Spannungsbereich vorgeben muss, sondern dass eine exakt zu erreichende Spannung angeben wird (Ein Toleranzbereich 'tol' besagt hierbei nur, ab wann das Ergebnis genau genug ist und keine weitere Reglerschleife nötig ist). Konkret heißt das, dass angenommen wird, dass die Stufen eines Trafos nicht ganzzahlich sein müssen. In unserem Beispiel können Sie bspw. diesen Regler für den zweiten Trafo anwenden. 

In [ ]:
trafo_controller = control.ContinuousTapControl(net=net, tid=142, vm_set_pu=0.98, tol=1e-6)

Wenn Sie nun einen Lastfluss mit aktivitierten Reglern durchführen, beträgt die Spannung unterspannungsseitig exakt 0.98 p.u.:

In [ ]:
pp.runpp(net, run_control=True)
net.res_trafo.vm_lv_pu

Des Weiteren, wie zu erwarten war, ist die Stufensteller-Position nicht mehr ganzzahlig, sondern liegt bei etwa -0.07:

In [ ]:
net.trafo['tap_pos']

Auch wenn dieses Result so nicht in der Realität vorkommen kann, so kann es dennoch nüzlich sein, um bspw. in groß angelegten Studien große Ergebnissprünge zu vermeiden.

## Lessons learned

Nach Abschluss dieses Tutorials:
- haben Sie den Aufbau des Control-Moduls verstanden.
- wissen Sie, welches die zentralen Bestandteile eines jeden Relgers sind.
- kennen Sie den Unterschied zwischen level und order.
- wissen Sie, welche Gefahren von recycle ausgehen.
- können Sie einen einfachen Stufenregler in ein Netz einbetten. 

## Aufgabe
1. Gehen Sie wieder vom MS Oberrhein-Netz aus. Bauen Sie nun einen zweiten zusätzlichen diskreten Stufenregler an den ersten Trafo mit den Grenzen (0.98, 0.995) ein. Was fällt Ihnen auf?

2. Überlegen Sie sich eine Methode, wie Sie das Problem aus 1. lösen können. Beiden Reglern sollen weiterhin die gleichen Grenzen vorgegeben sein. Wichtig hierbei: Die Entscheidung von Regler 2 sind denen von Regler 1 vorzuziehen.
